In [1]:
import open3d
import numpy as np
import pandas as pd
import os
import tqdm
import copy
import glob

import matplotlib.pyplot as plt
import utils.registration as registration
import utils.functions as functions
import utils.transform as transform
import utils.pointcloud as pointcloud
import utils.fread as fread
import utils.FCGF as FCGF

from utils.config import Config
from scipy.ndimage import gaussian_filter1d

In [2]:
dataset_dir = "data/raw_data"
features_dir = "data/features"
experiment = "exp_13"
calib_period = 4

In [5]:
for trial in os.listdir(os.path.join(dataset_dir, experiment)):
    
    if not trial.startswith("trial"):
        continue
    
    for subject in os.listdir(os.path.join(dataset_dir, experiment, trial, "secondary")):
        for sequence in os.listdir(os.path.join(dataset_dir, experiment, trial, "secondary", subject)):
            print(f"Processing > {trial} {sequence}")
            motion_dir = os.path.join(dataset_dir, experiment, trial, "secondary", subject, sequence, "motion")
            sequence_dir = os.path.join(dataset_dir, experiment, trial, "secondary", subject, sequence, "frames")
            feature_dir = os.path.join(features_dir, experiment, trial, "0.08", subject, sequence)

            sequence_ts = fread.get_timstamps_from_images(sequence_dir, ext=".depth.png")

            accel_df = pd.read_csv(os.path.join(motion_dir, "accel.csv"))
            gyro_df = pd.read_csv(os.path.join(motion_dir, "gyro.csv"))

            accel_df.drop_duplicates("timestamp", inplace=True)
            gyro_df.drop_duplicates("timestamp", inplace=True)
            imu_df = pd.merge(accel_df, gyro_df, on="timestamp", suffixes=("a", "g"))

            frame_rate = accel_df.shape[0] / (accel_df.timestamp.values[-1] - accel_df.timestamp.values[0]) * 1000
            win_len = int(frame_rate * calib_period)

            # remove first row as the dt is 0
            imu_df = imu_df.iloc[1:]
            # reset index in pandas data frame
            imu_df.reset_index(drop=True, inplace=True)

            imu_df = imu_df.iloc[win_len:].copy()

            # load ground truth trajectory
            start_t = np.where(sequence_ts == functions.nearest(sequence_ts, imu_df.timestamp.values[0]))[0][0]

            sequence_ts = sequence_ts[start_t:]
            for t in sequence_ts[:start_t]:
                if os.path.exists(os.path.join(sequence_dir, f"frame-{t}.depth.png")):
                    os.remove(os.path.join(sequence_dir, f"frame-{t}.depth.png"))
                if os.path.exists(os.path.join(sequence_dir, f"frame-{t}.color.png")):
                    os.remove(os.path.join(sequence_dir, f"frame-{t}.color.png"))
                if os.path.exists(os.path.join(feature_dir, f"{t}.global.npz")):
                    os.remove(os.path.join(feature_dir, f"{t}.global.npz"))
                if os.path.exists(os.path.join(feature_dir, f"{t}.secondary.npz")):
                    os.remove(os.path.join(feature_dir, f"{t}.secondary.npz"))    

Processing > trial_1 01
Processing > trial_1 02
Processing > trial_1 03
Processing > trial_1 04
Processing > trial_1 05
Processing > trial_1 06
Processing > trial_2 01
Processing > trial_2 02
Processing > trial_2 03
Processing > trial_2 04
Processing > trial_2 05
Processing > trial_2 06
Processing > trial_3 01
Processing > trial_3 02
Processing > trial_3 03
Processing > trial_3 04
Processing > trial_3 05
Processing > trial_3 06
Processing > trial_4 01
Processing > trial_4 02
Processing > trial_4 03
Processing > trial_4 04
Processing > trial_4 05
Processing > trial_4 06
Processing > trial_4 07


In [18]:
sequence_dirs = glob.glob("data/features/exp_12/trial_1/0.05/subject-1/*")

for sequence_dir in sequence_dirs:
    # sequence_ts = fread.get_timstamps_from_images(sequence_dir, ext=".depth.png")
    sequence_ts = fread.get_timstamps(sequence_dir, ext=".secondary.npz")

    duration = (sequence_ts[-1] - sequence_ts[0]) / 1000
    print(f"Duration: {duration} seconds")

Duration: 23.429 seconds
Duration: 23.495 seconds
Duration: 23.46 seconds
Duration: 23.494 seconds
Duration: 23.494 seconds


In [21]:
# sequence_dirs = glob.glob("data/point_clouds/exp_12/trial_1/subject-1/*")
sequence_dirs = glob.glob("data/features/exp_12/trial_1/0.03/subject-1/*")

for sequence_dir in sequence_dirs:
    # sequence_ts = fread.get_timstamps_from_images(sequence_dir, ext=".depth.png")
    sequence_ts = fread.get_timstamps(sequence_dir, ext=".secondary.npz")

    delta = np.diff(sequence_ts)
    fps = 1000 / np.mean(delta)
    
    print(f"FPS: {fps}")

FPS: 27.743394937897477
FPS: 27.79314747818685
FPS: 26.08695652173913
FPS: 27.70920234953605
FPS: 27.70920234953605


In [2]:
fragments = glob.glob("data/fragments/exp_13/*.pcd")

for fragment in fragments:
    pcd = open3d.io.read_point_cloud(fragment)
    open3d.visualization.draw_geometries([pcd])

KeyboardInterrupt: 

In [24]:
len(fragments)

50